<a href="https://colab.research.google.com/github/RonitGandhi/Hacksprint_PS01_InputOutput/blob/model_branch/TWEET_SCRAPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import datetime
import time
import pandas as pd
import tweepy

In [75]:
tweepy.__version__

'3.10.0'

In [76]:
def get_df():
    return pd.DataFrame(
        columns=[
            "tweet_id",
            "name",
            "screen_name",
            "retweet_count",
            "text",
            "mined_at",
            "created_at",
            "favourite_count",
            "hashtags",
            "status_count",
            "followers_count",
            "location",
            "source_device",
        ]
    )

In [77]:
class TweetMiner(object):

    result_limit = 20
    data = []
    api = False

    twitter_keys = {
        "consumer_key": "waFFtgAFbqBKyCdZlZCaNqosW",
        "consumer_secret": "nBcUonWTuzzN1YExrShinYQNrDe0x5cHdPavmhQv0Cs7MC8avN",
        "access_token_key": "1446707378657710081-3aBvyoS7zBhaJ4LQcKZyPjGRZSuwZ5",
        "access_token_secret": "YWoP4ZFdnNhjm4rceQMKF6Jc5LuRFJ6F24NvdZHYS3oqA",
    }
    def __init__(self, keys_dict=twitter_keys, api=api):

        self.twitter_keys = keys_dict

        auth = tweepy.OAuthHandler(
            keys_dict["consumer_key"], keys_dict["consumer_secret"]
        )
        auth.set_access_token(
            keys_dict["access_token_key"], keys_dict["access_token_secret"]
        )

        self.api = tweepy.API(
            auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True
        )
        self.twitter_keys = keys_dict
    def mine_crypto_currency_tweets(self, query="BTC"):

            last_tweet_id = False
            page_num = 1

            data = get_df()
            cypto_query = f"#{query}"
            print(" ===== ", query, cypto_query)
            for page in tweepy.Cursor(
                self.api.search,
                q=cypto_query,
                lang="en",
                tweet_mode="extended",
                count=200,  # max_id=1295144957439690000
            ).pages():
                print(" ...... new page", page_num)
                page_num += 1

                for item in page:
                    mined = {
                        "tweet_id": item.id,
                        "name": item.user.name,
                        "screen_name": item.user.screen_name,
                        "retweet_count": item.retweet_count,
                        "text": item.full_text,
                        "mined_at": datetime.datetime.now(),
                        "created_at": item.created_at,
                        "favourite_count": item.favorite_count,
                        "hashtags": item.entities["hashtags"],
                        "status_count": item.user.statuses_count,
                        "followers_count": item.user.followers_count,
                        "location": item.place,
                        "source_device": item.source,
                    }
                    try:
                        mined["retweet_text"] = item.retweeted_status.full_text
                    except:
                        mined["retweet_text"] = "None"

                    last_tweet_id = item.id
                    data = data.append(mined, ignore_index=True)

                if page_num % 180 == 0:
                    date_label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    print("....... outputting to csv", page_num, len(data))
                    data.to_csv(f"{query}_{page_num}_{date_label}.csv", index=False)
                    print("  ..... resetting df")
                    data = get_df()

            date_label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            data.to_csv(f"{query}_{page_num}_{date_label}.csv", index=False)

In [78]:
miner = TweetMiner()

In [79]:
handle_list = [
    "BTC",
    "ETH",
    "ADA",
    "BNB",
    "USDT",
    "XRP",
    "SOL",
    "DOT",
    "USDC",
    "DOGE"
]

In [80]:
import queue
import threading

should_publish = threading.Event()
update_queue = queue.Queue()


def start_publisher():
    global handle_list

    starttime = time.time()
    print("Start polling", starttime)
    poll_iteration = 1

    for i in range(10):
        for name in handle_list:
            print(i, poll_iteration, "\rpublishing update ", end="")
            update_queue.put((poll_iteration, name))
            poll_iteration += 1
            time.sleep(900)
            print("\rawaiting for publishing update", end="")
            should_publish.wait()
            update_queue.join()

def start_update_listener():
    while True:
        poll_iteration, name = update_queue.get()

        print(" --- ", name)
        try:
            miner.mine_crypto_currency_tweets(query=name)
            update_queue.task_done()

        except Exception as e:  # work on python 3.x
            print("Failed to upload to ftp: " + str(e))


listener_thread = threading.Thread(target=start_update_listener, daemon=True)
publisher_thread = threading.Thread(target=start_publisher, daemon=True)       

In [81]:
publisher_thread.start()
listener_thread.start()
# start publishing
should_publish.set()

Start polling 1633759491.1466806
publishing update  ---  BTC
 =====  BTC #BTC


In [82]:
should_publish.clear()